In [11]:
import sys
from __future__ import division
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import pandas as pd
from collections import Counter
from nltk.corpus import stopwords
import string
from matplotlib import rcParams
from itertools import chain
from nltk import everygrams, word_tokenize
import pprint
import enchant
stemmer = SnowballStemmer("english")
from pprint import pprint


# Create the datatset from which we will extract the BoW for LDA

In [12]:
df_train = pd.read_csv('../kaggle_data_exploration/all/train.csv', encoding = "utf_8")
#df_test = pd.read_csv('../kaggle_data_exploration/all/test.csv', encoding = 'utf_8')

In [13]:
pprint(df_train[:1])

                    qid                                      question_text  \
0  00002165364db923c7e6  How did Quebec nationalists see their province...   

   target  
0       0  


In [14]:
#df_train = pd.read_csv('../kaggle_data_exploration/all/train.csv', encoding = "utf_8")
#df_train = pd.read_csv('../kaggle_data_exploration/all/train.csv', encoding = 'utf_8')
data_t = df_train[['question_text']]
data_t['qid'] = df_train.qid
documents = data_t

print('documents length {} \n{}'.format(len(documents), documents[:5]))

documents length 1306122 
                                       question_text                   qid
0  How did Quebec nationalists see their province...  00002165364db923c7e6
1  Do you have an adopted dog, how would you enco...  000032939017120e6e44
2  Why does velocity affect time? Does velocity a...  0000412ca6e4628ce2cf
3  How did Otto von Guericke used the Magdeburg h...  000042bf85aa498cd78e
4  Can I convert montra helicon D to a mountain b...  0000455dfa3e01eae3af


/home/x/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [15]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

### Preview Document after preprocessing

In [16]:
doc_sample = documents[documents['qid']=='00002165364db923c7e6']
print(doc_sample)
print('\n')
doc_sample2 = documents.loc[0]
print(doc_sample2)

                                       question_text                   qid
0  How did Quebec nationalists see their province...  00002165364db923c7e6


question_text    How did Quebec nationalists see their province...
qid                                           00002165364db923c7e6
Name: 0, dtype: object


In [17]:
doc_sample = doc_sample.values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['How', 'did', 'Quebec', 'nationalists', 'see', 'their', 'province', 'as', 'a', 'nation', 'in', 'the', '1960s?']


 tokenized and lemmatized document: 
['quebec', 'nationalist', 'provinc', 'nation']


In [18]:
#preprocess the whole set
documents['question_text'][0]

'How did Quebec nationalists see their province as a nation in the 1960s?'

In [19]:
processed_docs = documents['question_text'].map(preprocess)
processed_docs[1:10]

1                [adopt, encourag, peopl, adopt, shop]
2    [veloc, affect, time, veloc, affect, space, ge...
3                [otto, guerick, magdeburg, hemispher]
4    [convert, montra, helicon, mountain, bike, cha...
5    [gaza, slowli, auschwitz, dachau, treblinka, p...
6    [quora, automat, conserv, opinion, report, lib...
7                   [crazi, wash, wipe, groceri, germ]
8         [thing, dress, moder, differ, dress, modest]
9    [phase, ignor, peopl, love, complet, disregard...
Name: question_text, dtype: object

In [20]:
print(len(processed_docs))
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 20:
        break

1306122
0 nation
1 nationalist
2 provinc
3 quebec
4 adopt
5 encourag
6 peopl
7 shop
8 affect
9 geometri
10 space
11 time
12 veloc
13 guerick
14 hemispher
15 magdeburg
16 otto
17 bike
18 chang
19 convert
20 helicon


In [21]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=10000)

In [22]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[1]

[(4, 2), (5, 1), (6, 1), (7, 1)]

In [23]:
bow_doc_1 = bow_corpus[1]
for i in range(len(bow_doc_1)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_1[i][0], 
                                               dictionary[bow_doc_1[i][0]], 
bow_doc_1[i][1]))

Word 4 ("adopt") appears 2 time.
Word 5 ("encourag") appears 1 time.
Word 6 ("peopl") appears 1 time.
Word 7 ("shop") appears 1 time.


In [24]:
    lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [25]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.074*"best" + 0.018*"import" + 0.017*"place" + 0.013*"way" + 0.012*"good" + 0.012*"term" + 0.012*"travel" + 0.012*"build" + 0.011*"food" + 0.010*"increas"
Topic: 1 
Words: 0.019*"caus" + 0.015*"happen" + 0.015*"social" + 0.013*"parent" + 0.013*"water" + 0.013*"stop" + 0.013*"effect" + 0.012*"lose" + 0.010*"bodi" + 0.009*"media"
Topic: 2 
Words: 0.027*"quora" + 0.023*"question" + 0.023*"write" + 0.023*"book" + 0.016*"movi" + 0.015*"answer" + 0.014*"read" + 0.014*"game" + 0.013*"play" + 0.010*"follow"
Topic: 3 
Words: 0.022*"compani" + 0.018*"start" + 0.018*"money" + 0.015*"busi" + 0.015*"onlin" + 0.014*"market" + 0.013*"number" + 0.013*"account" + 0.012*"product" + 0.010*"india"
Topic: 4 
Words: 0.023*"friend" + 0.023*"love" + 0.019*"human" + 0.016*"languag" + 0.015*"have" + 0.013*"year" + 0.012*"like" + 0.012*"relationship" + 0.012*"state" + 0.011*"tell"
Topic: 5 
Words: 0.026*"work" + 0.013*"problem" + 0.010*"hous" + 0.010*"face" + 0.010*"time" + 0.008*"function" + 0

# trying the model with some instances...

In [26]:
processed_docs[1]

['adopt', 'encourag', 'peopl', 'adopt', 'shop']

In [27]:
for index, score in sorted(lda_model[bow_corpus[1]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.5041003823280334	 
Topic: 0.054*"peopl" + 0.033*"like" + 0.029*"person" + 0.029*"life" + 0.025*"mean" + 0.018*"feel" + 0.016*"american" + 0.016*"women" + 0.016*"look" + 0.014*"want"

Score: 0.19589513540267944	 
Topic: 0.024*"thing" + 0.022*"world" + 0.022*"countri" + 0.020*"india" + 0.016*"think" + 0.016*"trump" + 0.015*"peopl" + 0.015*"go" + 0.012*"indian" + 0.011*"muslim"

Score: 0.18333394825458527	 
Topic: 0.022*"compani" + 0.018*"start" + 0.018*"money" + 0.015*"busi" + 0.015*"onlin" + 0.014*"market" + 0.013*"number" + 0.013*"account" + 0.012*"product" + 0.010*"india"

Score: 0.01667051762342453	 
Topic: 0.074*"best" + 0.018*"import" + 0.017*"place" + 0.013*"way" + 0.012*"good" + 0.012*"term" + 0.012*"travel" + 0.012*"build" + 0.011*"food" + 0.010*"increas"

Score: 0.01666666753590107	 
Topic: 0.019*"caus" + 0.015*"happen" + 0.015*"social" + 0.013*"parent" + 0.013*"water" + 0.013*"stop" + 0.013*"effect" + 0.012*"lose" + 0.010*"bodi" + 0.009*"media"

Score: 0.016666667535

# trying the model with all training instances

In [95]:
import itertools
import operator

def averageTL(tuplelist):
    d = {}
    for el in tuplelist:
        for tuple in el:
            key,val = tuple
            d.setdefault(key, []).append(val)
     
    return(d) 
    
    
    
#mec = [[(0, 0.016670508), (1, 0.016666668), (2, 0.016666668), (3, 0.18333398), (4, 0.016666668), (5, 0.016666668), (6, 0.016666668), (7, 0.016666668), (8, 0.50409806), (9, 0.19589746)], [(0, 0.012500407), (1, 0.28031057), (2, 0.012500324), (3, 0.012500497), (4, 0.23546916), (5, 0.26042563), (6, 0.14879261), (7, 0.012500024), (8, 0.012500535), (9, 0.012500283)], [(0, 0.7), (1, 0.033333335), (2, 0.033333335), (3, 0.033333335), (4, 0.033333335), (5, 0.033333335), (6, 0.033333335), (7, 0.033333335), (8, 0.033333335), (9, 0.033333335)], [(0, 0.016666668), (1, 0.016679456), (2, 0.016668567), (3, 0.51121813), (4, 0.016668288), (5, 0.016671726), (6, 0.016667815), (7, 0.01666667), (8, 0.016670717), (9, 0.35542193)], [(0, 0.025), (1, 0.025), (2, 0.025001911), (3, 0.025), (4, 0.025), (5, 0.275), (6, 0.025), (7, 0.025), (8, 0.5249981), (9, 0.025)], [(0, 0.0125), (1, 0.0125), (2, 0.28289184), (3, 0.012503062), (4, 0.0125), (5, 0.13749853), (6, 0.0125), (7, 0.012501531), (8, 0.0125003345), (9, 0.4921047)], [(0, 0.016666668), (1, 0.183333), (2, 0.016666668), (3, 0.016666668), (4, 0.016666668), (5, 0.18333335), (6, 0.016666668), (7, 0.35000035), (8, 0.016666668), (9, 0.1833333)], [(0, 0.014285714), (1, 0.014285714), (2, 0.014285714), (3, 0.44285735), (4, 0.014285714), (5, 0.014285714), (6, 0.1571425), (7, 0.15714295), (8, 0.014285714), (9, 0.15714289)], [(3, 0.15684174), (4, 0.08903002), (7, 0.07857286), (8, 0.557643), (9, 0.08219377)]]
#averageTL(mec,1)


In [102]:
def processDfAndAverage(df, LDAModel):
    texts = df['question_text']
    print('len {}\n{}'.format(len(texts), texts[:5]))
    preprocessed_texts = [preprocess(text) for text in texts]
    print('len {}\n{}'.format(len(preprocessed_texts), preprocessed_texts[:5]))
    bow_vectors = [dictionary.doc2bow(text) for text in preprocessed_texts]
    print('len {}\n{}'.format(len(bow_vectors), bow_vectors[:5]))
    lda_results = [lda_model[bow] for bow in bow_vectors]
    print('len {}\n{}'.format(len(lda_results), lda_results[:10]))
    print('RESULTS')
    
    atl = averageTL(lda_results)
    for name, values in atl.items():
        print("{name} {avg}".format(name=name, avg=sum(values)/len(lda_results)))
    

In [107]:
sub_0 = df_train.loc[df_train['target']==0][:100000]
print(len(sub_0))
processDfAndAverage(sub_0, lda_model)

100000
len 100000
0    How did Quebec nationalists see their province...
1    Do you have an adopted dog, how would you enco...
2    Why does velocity affect time? Does velocity a...
3    How did Otto von Guericke used the Magdeburg h...
4    Can I convert montra helicon D to a mountain b...
Name: question_text, dtype: object
len 100000
[['quebec', 'nationalist', 'provinc', 'nation'], ['adopt', 'encourag', 'peopl', 'adopt', 'shop'], ['veloc', 'affect', 'time', 'veloc', 'affect', 'space', 'geometri'], ['otto', 'guerick', 'magdeburg', 'hemispher'], ['convert', 'montra', 'helicon', 'mountain', 'bike', 'chang', 'tyre']]
len 100000
[[(0, 1), (1, 1), (2, 1), (3, 1)], [(4, 2), (5, 1), (6, 1), (7, 1)], [(8, 2), (9, 1), (10, 1), (11, 1), (12, 2)], [(13, 1), (14, 1)], [(15, 1), (16, 1), (17, 1), (18, 1), (19, 1)]]
len 100000
[[(0, 0.02), (1, 0.02), (2, 0.02), (3, 0.02), (4, 0.41997617), (5, 0.21999998), (6, 0.02), (7, 0.22000027), (8, 0.02), (9, 0.020023575)], [(0, 0.016670514), (1, 0.016666668)

In [108]:
sub_1 = df_train.loc[df_train['target']==1]
print(len(sub_1))
processDfAndAverage(sub_1, lda_model)

80810
len 80810
22     Has the United States become the largest dicta...
30     Which babies are more sweeter to their parents...
110    If blacks support school choice and mandatory ...
114    I am gay boy and I love my cousin (boy). He is...
115                 Which races have the smallest penis?
Name: question_text, dtype: object
len 80810
[['unit', 'state', 'largest', 'dictatorship', 'world'], ['babi', 'sweeter', 'parent', 'dark', 'skin', 'babi', 'light', 'skin', 'babi'], ['black', 'support', 'school', 'choic', 'mandatori', 'sentenc', 'crimin', 'vote', 'republican'], ['love', 'cousin', 'sexi', 'dont', 'know', 'want'], ['race', 'smallest', 'peni']]
len 80810
[[(96, 1), (97, 1), (98, 1), (99, 1), (100, 1)], [(77, 2), (131, 3), (132, 1), (133, 1), (134, 1)], [(135, 1), (175, 1), (432, 1), (433, 1), (434, 1), (435, 1), (436, 1), (437, 1), (438, 1)], [(46, 1), (68, 1), (449, 1), (450, 1), (451, 1), (452, 1)], [(453, 1), (454, 1), (455, 1)]]
len 80810
[[(0, 0.016666668), (1, 0.016666668

In [110]:
lda_model.print_topics()

[(0,
  '0.074*"best" + 0.018*"import" + 0.017*"place" + 0.013*"way" + 0.012*"good" + 0.012*"term" + 0.012*"travel" + 0.012*"build" + 0.011*"food" + 0.010*"increas"'),
 (1,
  '0.019*"caus" + 0.015*"happen" + 0.015*"social" + 0.013*"parent" + 0.013*"water" + 0.013*"stop" + 0.013*"effect" + 0.012*"lose" + 0.010*"bodi" + 0.009*"media"'),
 (2,
  '0.027*"quora" + 0.023*"question" + 0.023*"write" + 0.023*"book" + 0.016*"movi" + 0.015*"answer" + 0.014*"read" + 0.014*"game" + 0.013*"play" + 0.010*"follow"'),
 (3,
  '0.022*"compani" + 0.018*"start" + 0.018*"money" + 0.015*"busi" + 0.015*"onlin" + 0.014*"market" + 0.013*"number" + 0.013*"account" + 0.012*"product" + 0.010*"india"'),
 (4,
  '0.023*"friend" + 0.023*"love" + 0.019*"human" + 0.016*"languag" + 0.015*"have" + 0.013*"year" + 0.012*"like" + 0.012*"relationship" + 0.012*"state" + 0.011*"tell"'),
 (5,
  '0.026*"work" + 0.013*"problem" + 0.010*"hous" + 0.010*"face" + 0.010*"time" + 0.008*"function" + 0.008*"phone" + 0.008*"project" + 0.007*

# Adding a new row and saving the csv file

In [160]:

df1 = df_train
df1 = df1.assign(e=pd.Series(np.random.randn(len(df1))).values)
print(df1.head())
print(df1.tail())
df1.to_csv('test.csv', sep='\t', encoding='utf-8')


                    qid                                      question_text  \
0  00002165364db923c7e6  How did Quebec nationalists see their province...   
1  000032939017120e6e44  Do you have an adopted dog, how would you enco...   
2  0000412ca6e4628ce2cf  Why does velocity affect time? Does velocity a...   
3  000042bf85aa498cd78e  How did Otto von Guericke used the Magdeburg h...   
4  0000455dfa3e01eae3af  Can I convert montra helicon D to a mountain b...   

   target         e  
0       0  1.389152  
1       0  1.326635  
2       0  2.257668  
3       0 -0.029960  
4       0  0.413381  
                          qid  \
1306117  ffffcc4e2331aaf1e41e   
1306118  ffffd431801e5a2f4861   
1306119  ffffd48fb36b63db010c   
1306120  ffffec519fa37cf60c78   
1306121  ffffed09fedb5088744a   

                                             question_text  target         e  
1306117  What other technical skills do you need as a c...       0  0.524521  
1306118  Does MS in ECE have good job pros

In [125]:
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.3499702513217926	 Topic: 0.049*"like" + 0.030*"peopl" + 0.025*"feel" + 0.024*"know" + 0.023*"person"
Score: 0.18336229026317596	 Topic: 0.034*"countri" + 0.030*"world" + 0.023*"state" + 0.015*"american" + 0.014*"peopl"
Score: 0.1833333969116211	 Topic: 0.027*"learn" + 0.018*"develop" + 0.017*"mean" + 0.014*"compani" + 0.012*"product"
Score: 0.1833331435918808	 Topic: 0.080*"best" + 0.018*"india" + 0.015*"effect" + 0.013*"place" + 0.012*"water"
Score: 0.016667576506733894	 Topic: 0.029*"think" + 0.023*"peopl" + 0.020*"trump" + 0.019*"indian" + 0.014*"right"
Score: 0.01666666753590107	 Topic: 0.033*"work" + 0.026*"engin" + 0.018*"student" + 0.017*"colleg" + 0.014*"studi"
Score: 0.01666666753590107	 Topic: 0.043*"good" + 0.031*"book" + 0.025*"best" + 0.022*"money" + 0.020*"write"
Score: 0.01666666753590107	 Topic: 0.026*"quora" + 0.022*"life" + 0.022*"question" + 0.019*"chang" + 0.015*"answer"
Score: 0.01666666753590107	 Topic: 0.018*"import" + 0.015*"class" + 0.015*"univers" + 0